# Risk Sensitivity Analysis

Deze notebook is voor het uitvoeren van een SA op risk weging.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from SALib.analyze import sobol
from SALib.sample import saltelli 

from model_find_paths import connect_distribution_to_postnl

# Algemene risks

In [2]:
df = pd.read_csv("/Users/cmartens/Documents/thesis_cf_martens/2.risk_analysis/input/risk_scores.csv") 
# drop last 3 rows 
df = df.drop(df.index[-3:])


In [3]:
df

,area_type,Sf,Sp,Ss,L1,L2,L3,L4,L5,Height
0,Motorways and major roads,4.0,3.0,1.0,3.0,2.0,1.0,1.0,3.0,30.0
1,Regional roads,3.0,3.0,2.0,4.0,3.0,1.0,3.0,2.0,30.0
2,Tracks and rural access roads,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,30.0
3,Living and residential streets,3.0,4.0,4.0,5.0,4.0,1.0,5.0,3.0,30.0
4,Pedestrian and cycling paths,3.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,30.0
5,Railways,4.0,3.0,2.0,5.0,2.0,2.0,1.0,4.0,30.0
6,Power lines,1.0,4.0,1.0,2.0,2.0,3.0,1.0,5.0,60.0
7,Power plants,1.0,4.0,2.0,4.0,3.0,2.0,3.0,5.0,60.0
8,Communication towers,1.0,3.0,1.0,4.0,5.0,1.0,2.0,4.0,60.0
9,High infrastructures,1.0,3.0,2.0,4.0,2.0,1.0,3.0,3.0,60.0


In [4]:
# Aantal externe risicofactoren
n_factors = 5

# Bereken cumulatieve bijdrage per domein (zonder deling)
R_f_total = 0
R_p_total = 0

# Loop over externe risicofactoren
for i in range(1, n_factors + 1):
    # Bereken afzonderlijke fatality- en property-risicobijdragen
    df[f"R_if_{i}"] = df["Sf"] * df[f"L{i}"]
    df[f"R_ip_{i}"] = df["Sp"] * df[f"L{i}"]
    
    # Tel cumulatief op
    R_f_total += df[f"R_if_{i}"]
    R_p_total += df[f"R_ip_{i}"]

# Cumulatieve crash-gerelateerde risico's
df["R_f"] = R_f_total
df["R_p"] = R_p_total

# Sociaal risico (losstaand van risicofactoren)
df["R_s"] = df["Ss"]

# Normaliseer fatality, property en societal domeinen afzonderlijk
df["R_f_norm"] = (df["R_f"] - df["R_f"].min()) / (df["R_f"].max() - df["R_f"].min())
df["R_p_norm"] = (df["R_p"] - df["R_p"].min()) / (df["R_p"].max() - df["R_p"].min())
df["R_s_norm"] = (df["R_s"] - df["R_s"].min()) / (df["R_s"].max() - df["R_s"].min())

In [5]:
df

,area_type,Sf,Sp,Ss,L1,L2,L3,L4,L5,Height,...,R_if_4,R_ip_4,R_if_5,R_ip_5,R_f,R_p,R_s,R_f_norm,R_p_norm,R_s_norm
0,Motorways and major roads,4.0,3.0,1.0,3.0,2.0,1.0,1.0,3.0,30.0,...,4.0,3.0,12.0,9.0,40.0,30.0,1.0,0.548387,0.307692,0.000000
1,Regional roads,3.0,3.0,2.0,4.0,3.0,1.0,3.0,2.0,30.0,...,9.0,9.0,6.0,6.0,39.0,39.0,2.0,0.532258,0.423077,0.333333
2,Tracks and rural access roads,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,30.0,...,4.0,4.0,4.0,4.0,20.0,20.0,1.0,0.225806,0.179487,0.000000
3,Living and residential streets,3.0,4.0,4.0,5.0,4.0,1.0,5.0,3.0,30.0,...,15.0,20.0,9.0,12.0,54.0,72.0,4.0,0.774194,0.846154,1.000000
4,Pedestrian and cycling paths,3.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,30.0,...,6.0,4.0,6.0,4.0,36.0,24.0,3.0,0.483871,0.230769,0.666667
5,Railways,4.0,3.0,2.0,5.0,2.0,2.0,1.0,4.0,30.0,...,4.0,3.0,16.0,12.0,56.0,42.0,2.0,0.806452,0.461538,0.333333
6,Power lines,1.0,4.0,1.0,2.0,2.0,3.0,1.0,5.0,60.0,...,1.0,4.0,5.0,20.0,13.0,52.0,1.0,0.112903,0.589744,0.000000
7,Power plants,1.0,4.0,2.0,4.0,3.0,2.0,3.0,5.0,60.0,...,3.0,12.0,5.0,20.0,17.0,68.0,2.0,0.177419,0.794872,0.333333
8,Communication towers,1.0,3.0,1.0,4.0,5.0,1.0,2.0,4.0,60.0,...,2.0,6.0,4.0,12.0,16.0,48.0,1.0,0.161290,0.538462,0.000000
9,High infrastructures,1.0,3.0,2.0,4.0,2.0,1.0,3.0,3.0,60.0,...,3.0,9.0,3.0,9.0,13.0,39.0,2.0,0.112903,0.423077,0.333333


# SA1: Sensitivity on severity properties

In [13]:
import pandas as pd
import numpy as np
import copy

# Gebiedstypes voor SA
target_area_types = [
    "Meadows and open grass", 
    "Pedestrian and cycling paths", 
    "Regional roads", 
    "Rivers, canals and streams", 
    "Tracks and rural access roads"
]

# Severity kolommen (fatality, property, societal)
severity_cols = ["Sf", "Sp", "Ss"]
severity_names = ["fatality", "property", "societal"]

# Alpha-waarden om te testen
alpha_values = [0.0, 0.5, 1.0]

# Opslagstructuur
sa_results = []

for area in target_area_types:
    for i, severity_col in enumerate(severity_cols):
        severity_name = severity_names[i]
        for delta in [-1, +1]:
            scenario_name = f"{area} - {severity_name} {'+' if delta > 0 else ''}{delta}"

            # Deepcopy van df om origineel te behouden
            df_mod = df.copy()

            # Pas de severity waarde alleen aan voor het geselecteerde gebiedstype
            affected_rows = df_mod["area_type"] == area
            df_mod.loc[affected_rows, severity_col] = (
                df_mod.loc[affected_rows, severity_col] + delta
            ).clip(lower=1, upper=4)  # Severity grenzen: 1=Negligible, 4=Catastrophic

            # Herbereken risico's met aangepaste severity
            R_f_total, R_p_total = 0, 0
            for j in range(1, 6):  # Loop over likelihood factors L1-L5
                df_mod[f"R_if_{j}"] = df_mod["Sf"] * df_mod[f"L{j}"]
                df_mod[f"R_ip_{j}"] = df_mod["Sp"] * df_mod[f"L{j}"]
                R_f_total += df_mod[f"R_if_{j}"]
                R_p_total += df_mod[f"R_ip_{j}"]

            df_mod["R_f"] = R_f_total
            df_mod["R_p"] = R_p_total
            df_mod["R_s"] = df_mod["Ss"]  # Societal risk = severity only

            # Normalisatie
            df_mod["R_f_norm"] = (df_mod["R_f"] - df_mod["R_f"].min()) / (df_mod["R_f"].max() - df_mod["R_f"].min())
            df_mod["R_p_norm"] = (df_mod["R_p"] - df_mod["R_p"].min()) / (df_mod["R_p"].max() - df_mod["R_p"].min())
            df_mod["R_s_norm"] = (df_mod["R_s"] - df_mod["R_s"].min()) / (df_mod["R_s"].max() - df_mod["R_s"].min())

            # Loop over alpha's
            for alpha in alpha_values:
                # Gewogen risicoscore toepassen
                df_mod["risk"] = (
                    0.4 * df_mod["R_f_norm"] + 
                    0.3 * df_mod["R_p_norm"] + 
                    0.3 * df_mod["R_s_norm"]
                )

                # Mapping area_type -> risk voor edges
                etype_to_risk = dict(zip(df_mod["area_type"], df_mod["risk"]))
                for u, v, data in G.edges(data=True):
                    etype = data.get("etype")
                    if etype in etype_to_risk:
                        data["risk"] = etype_to_risk[etype]
                    if etype == "postnl_connector":
                        data["risk"] = 0.0

                # Run planner
                connected, _ = connect_distribution_to_postnl(G, alpha=alpha, method="dijkstra")

                # Resultaatanalyse
                total_risk = 0
                total_energy = 0
                etype_usage = []
                total_length = 0
                for conn in connected:
                    _, _, _, _, path_edges, etypes = conn
                    etype_usage.extend(etypes)
                    for geom, u, v in path_edges:
                        edge = G.get_edge_data(u, v)
                        total_risk += edge.get("risk", 0)
                        total_energy += edge.get("energy", 0)
                        total_length += geom.length

                sa_results.append({
                    "scenario": scenario_name,
                    "alpha": alpha,
                    "n_connected": len(connected),
                    "total_risk": total_risk,
                    "total_energy": total_energy,
                    "total_length": total_length,
                    "etype_usage": etype_usage,
                })

# Convert to DataFrame for analysis
sa_df = pd.DataFrame(sa_results)
print(f"Completed {len(sa_results)} sensitivity scenarios")
print(sa_df.head())

Distribution points: 1
PostNL points: 64
Alpha: 0.0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m | Cost: 59.93
Connected: 157648 → 1 | Length: 5411.2 m | Cost: 143.23
Connected: 157648 → 2 | Length: 7091.7 m | Cost: 189.30
Connected: 157648 → 3 | Length: 4856.1 m | Cost: 129.35
Connected: 157648 → 4 | Length: 3481.7 m | Cost: 94.99
Connected: 157648 → 5 | Length: 3151.6 m | Cost: 90.80
Connected: 157648 → 6 | Length: 4250.9 m | Cost: 118.28
Connected: 157648 → 7 | Length: 2861.1 m | Cost: 79.48
Connected: 157648 → 8 | Length: 2643.8 m | Cost: 76.08
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m | Cost: 86.26
Connected: 157648 → 11 | Length: 5987.9 m | Cost: 161.71
Connected: 157648 → 12 | Length: 3133.2 m | Cost: 88.31
Connected: 157648 → 13 | Length: 5332.2 m | Cost: 145.31
Connected: 157648 → 14 | Length: 6446.6 m | Cost: 169.11
Connected: 157648 → 15 | Length: 6200.1 m | Cost: 162.95
Connected: 157648 → 16 | Length: 4979.6 m | Cost: 134.47
Connected: 157648 → 17 

In [14]:
import pandas as pd
import numpy as np

# First, let's get the baseline results for comparison
baseline_results = []
for alpha in alpha_values:
    # Run baseline scenario (no modifications)
    df_baseline = df.copy()  # Original df
    
    # Apply original risk calculations
    R_f_total, R_p_total = 0, 0
    for j in range(1, 6):
        df_baseline[f"R_if_{j}"] = df_baseline["Sf"] * df_baseline[f"L{j}"]
        df_baseline[f"R_ip_{j}"] = df_baseline["Sp"] * df_baseline[f"L{j}"]
        R_f_total += df_baseline[f"R_if_{j}"]
        R_p_total += df_baseline[f"R_ip_{j}"]
    
    df_baseline["R_f"] = R_f_total
    df_baseline["R_p"] = R_p_total
    df_baseline["R_s"] = df_baseline["Ss"]
    
    # Normalization
    df_baseline["R_f_norm"] = (df_baseline["R_f"] - df_baseline["R_f"].min()) / (df_baseline["R_f"].max() - df_baseline["R_f"].min())
    df_baseline["R_p_norm"] = (df_baseline["R_p"] - df_baseline["R_p"].min()) / (df_baseline["R_p"].max() - df_baseline["R_p"].min())
    df_baseline["R_s_norm"] = (df_baseline["R_s"] - df_baseline["R_s"].min()) / (df_baseline["R_s"].max() - df_baseline["R_s"].min())
    
    df_baseline["risk"] = (
        0.4 * df_baseline["R_f_norm"] + 
        0.3 * df_baseline["R_p_norm"] + 
        0.3 * df_baseline["R_s_norm"]
    )
    
    # Apply to graph and run planner
    etype_to_risk = dict(zip(df_baseline["area_type"], df_baseline["risk"]))
    for u, v, data in G.edges(data=True):
        etype = data.get("etype")
        if etype in etype_to_risk:
            data["risk"] = etype_to_risk[etype]
        if etype == "postnl_connector":
            data["risk"] = 0.0
    
    connected, _ = connect_distribution_to_postnl(G, alpha=alpha, method="dijkstra")
    
    # Calculate baseline metrics
    total_risk = sum(G.get_edge_data(u, v).get("risk", 0) 
                    for conn in connected 
                    for _, _, _, _, path_edges, _ in [conn]
                    for _, u, v in path_edges)
    total_energy = sum(G.get_edge_data(u, v).get("energy", 0) 
                      for conn in connected 
                      for _, _, _, _, path_edges, _ in [conn]
                      for _, u, v in path_edges)
    total_length = sum(geom.length 
                      for conn in connected 
                      for _, _, _, _, path_edges, _ in [conn]
                      for geom, u, v in path_edges)
    
    baseline_results.append({
        "scenario": "Baseline",
        "alpha": alpha,
        "n_connected": len(connected),
        "total_risk": total_risk,
        "total_energy": total_energy,
        "total_length": total_length
    })

baseline_df = pd.DataFrame(baseline_results)

# Now create analysis tables
def create_sensitivity_summary_table(sa_df, baseline_df):
    """Create summary table showing deviations from baseline"""
    
    # Parse scenario information
    sa_df_parsed = sa_df.copy()
    sa_df_parsed[['area_type', 'parameter_change']] = sa_df_parsed['scenario'].str.split(' - ', expand=True)
    sa_df_parsed[['parameter', 'change_direction']] = sa_df_parsed['parameter_change'].str.extract(r'(\w+)\s*([+-]\d+)')
    sa_df_parsed['change_value'] = sa_df_parsed['change_direction'].astype(int)
    
    # Merge with baseline
    merged = sa_df_parsed.merge(baseline_df[['alpha', 'total_risk', 'total_energy', 'total_length']], 
                               on='alpha', suffixes=('', '_baseline'))
    
    # Calculate percentage changes
    merged['risk_change_pct'] = ((merged['total_risk'] - merged['total_risk_baseline']) / merged['total_risk_baseline'] * 100)
    merged['energy_change_pct'] = ((merged['total_energy'] - merged['total_energy_baseline']) / merged['total_energy_baseline'] * 100)
    merged['length_change_pct'] = ((merged['total_length'] - merged['total_length_baseline']) / merged['total_length_baseline'] * 100)
    
    return merged

def create_summary_statistics_table(merged_df):
    """Create table with summary statistics for thesis"""
    
    summary_stats = []
    
    for alpha in alpha_values:
        alpha_data = merged_df[merged_df['alpha'] == alpha]
        
        # Overall statistics
        stats = {
            'alpha': alpha,
            'n_scenarios': len(alpha_data),
            'risk_change_mean': alpha_data['risk_change_pct'].mean(),
            'risk_change_std': alpha_data['risk_change_pct'].std(),
            'risk_change_max_abs': alpha_data['risk_change_pct'].abs().max(),
            'energy_change_mean': alpha_data['energy_change_pct'].mean(),
            'energy_change_std': alpha_data['energy_change_pct'].std(),
            'energy_change_max_abs': alpha_data['energy_change_pct'].abs().max(),
            'length_change_mean': alpha_data['length_change_pct'].mean(),
            'length_change_std': alpha_data['length_change_pct'].std(),
            'length_change_max_abs': alpha_data['length_change_pct'].abs().max(),
        }
        summary_stats.append(stats)
    
    return pd.DataFrame(summary_stats)

def create_area_parameter_table(merged_df):
    """Create table showing sensitivity by area type and parameter"""
    
    area_param_stats = []
    
    for alpha in alpha_values:
        alpha_data = merged_df[merged_df['alpha'] == alpha]
        
        for area in target_area_types:
            area_data = alpha_data[alpha_data['area_type'] == area]
            
            for param in ['fatality', 'property', 'societal']:
                param_data = area_data[area_data['parameter'] == param]
                
                if len(param_data) > 0:
                    stats = {
                        'alpha': alpha,
                        'area_type': area,
                        'parameter': param,
                        'risk_change_range': f"{param_data['risk_change_pct'].min():.2f} to {param_data['risk_change_pct'].max():.2f}",
                        'energy_change_range': f"{param_data['energy_change_pct'].min():.2f} to {param_data['energy_change_pct'].max():.2f}",
                        'length_change_range': f"{param_data['length_change_pct'].min():.2f} to {param_data['length_change_pct'].max():.2f}",
                        'max_abs_risk_change': param_data['risk_change_pct'].abs().max(),
                        'max_abs_energy_change': param_data['energy_change_pct'].abs().max(),
                        'max_abs_length_change': param_data['length_change_pct'].abs().max(),
                    }
                    area_param_stats.append(stats)
    
    return pd.DataFrame(area_param_stats)

def create_latex_table(df, caption, label):
    """Convert DataFrame to LaTeX table format"""
    
    # Round numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df_rounded = df.copy()
    df_rounded[numeric_cols] = df_rounded[numeric_cols].round(3)
    
    latex_str = df_rounded.to_latex(index=False, escape=False, column_format='l' * len(df.columns))
    
    # Add caption and label
    latex_str = latex_str.replace('\\begin{tabular}', 
                                 f'\\begin{{table}}[htbp]\n\\centering\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\begin{{tabular}}')
    latex_str = latex_str.replace('\\end{tabular}', '\\end{tabular}\n\\end{table}')
    
    return latex_str

# Run the analysis
print("Processing sensitivity analysis results...")
merged_results = create_sensitivity_summary_table(sa_df, baseline_df)

# Create summary tables
summary_stats = create_summary_statistics_table(merged_results)
area_param_stats = create_area_parameter_table(merged_results)

# Display results
print("\n=== SUMMARY STATISTICS BY ALPHA ===")
print(summary_stats.round(3))

print("\n=== TOP 10 LARGEST CHANGES ===")
top_changes = merged_results.nlargest(10, 'risk_change_pct')[['scenario', 'alpha', 'risk_change_pct', 'energy_change_pct', 'length_change_pct']]
print(top_changes.round(3))

print("\n=== AREA-PARAMETER SENSITIVITY (α=0.5) ===")
alpha_05_stats = area_param_stats[area_param_stats['alpha'] == 0.5][['area_type', 'parameter', 'max_abs_risk_change', 'max_abs_energy_change']]
print(alpha_05_stats.round(3))

# Generate LaTeX tables for thesis
print("\n=== LaTeX TABLES ===")
print("\n1. Summary Statistics Table:")
print(create_latex_table(summary_stats, 
                        "Sensitivity analysis summary statistics across α values", 
                        "tab:sa_severity_summary"))

print("\n2. Area-Parameter Table (α=0.5):")
alpha_05_table = area_param_stats[area_param_stats['alpha'] == 0.5][['area_type', 'parameter', 'max_abs_risk_change', 'max_abs_energy_change', 'max_abs_length_change']]
print(create_latex_table(alpha_05_table,
                        "Maximum absolute changes by area type and severity parameter (α=0.5)",
                        "tab:sa_severity_area_param"))

# Save detailed results
merged_results.to_csv('severity_sensitivity_detailed_results.csv', index=False)
summary_stats.to_csv('severity_sensitivity_summary.csv', index=False)
area_param_stats.to_csv('severity_sensitivity_area_param.csv', index=False)

print(f"\nAnalysis complete. Processed {len(sa_df)} sensitivity scenarios.")
print("Detailed results saved to CSV files.")

Distribution points: 1
PostNL points: 64
Alpha: 0.0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m | Cost: 59.93
Connected: 157648 → 1 | Length: 5411.2 m | Cost: 143.23
Connected: 157648 → 2 | Length: 7091.7 m | Cost: 189.30
Connected: 157648 → 3 | Length: 4856.1 m | Cost: 129.35
Connected: 157648 → 4 | Length: 3481.7 m | Cost: 94.99
Connected: 157648 → 5 | Length: 3151.6 m | Cost: 90.80
Connected: 157648 → 6 | Length: 4250.9 m | Cost: 118.28
Connected: 157648 → 7 | Length: 2861.1 m | Cost: 79.48
Connected: 157648 → 8 | Length: 2643.8 m | Cost: 76.08
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m | Cost: 86.26
Connected: 157648 → 11 | Length: 5987.9 m | Cost: 161.71
Connected: 157648 → 12 | Length: 3133.2 m | Cost: 88.31
Connected: 157648 → 13 | Length: 5332.2 m | Cost: 145.31
Connected: 157648 → 14 | Length: 6446.6 m | Cost: 169.11
Connected: 157648 → 15 | Length: 6200.1 m | Cost: 162.95
Connected: 157648 → 16 | Length: 4979.6 m | Cost: 134.47
Connected: 157648 → 17 

# SA 2: Sensitivity analysis on likelihood scores of external factors in frequently used corridors

In [6]:
city = 'breda'
depot = ['Breda']

import pickle
import networkx as nx

with open(f'/Users/cmartens/Documents/thesis_cf_martens/model/graph_creation/output/{city}.pkl', 'rb') as f:
    G = pickle.load(f)


In [7]:
import pandas as pd
import numpy as np
import copy

# Gebiedstypes voor SA
target_area_types = [
    "Meadows and open grass", 
    "Pedestrian and cycling paths", 
    "Regional roads", 
    "Rivers, canals and streams", 
    "Tracks and rural access roads"
]

# Likelihood kolommen
likelihood_cols = [f"L{i}" for i in range(1, 6)]

# Alpha-waarden om te testen
alpha_values = [0.0, 0.5, 1.0]

# Opslagstructuur
sa_results = []

In [8]:
# Base case gebruikt de risico's die al op de edges van G staan
for alpha in alpha_values:
    connected, _ = connect_distribution_to_postnl(G, alpha=alpha, method="dijkstra")

    total_risk = 0
    total_energy = 0
    total_length = 0
    etype_usage = []

    for conn in connected:
        _, _, path_length, _, path_edges, etypes = conn
        etype_usage.extend(etypes)
        total_length += path_length
        for geom, u, v in path_edges:
            edge = G.get_edge_data(u, v)
            length = edge.get("length", geom.length)
            risk = edge.get("risk", 0)
            energy = edge.get("energy", 0)

            total_risk += risk * length
            total_energy += energy

    sa_results.append({
        "scenario": "Base",
        "alpha": alpha,
        "n_connected": len(connected),
        "total_risk": total_risk,
        "total_energy": total_energy,
        "total_length": total_length,
        "etype_usage": etype_usage
    })

Distribution points: 1
PostNL points: 64
Alpha: 0.0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m | Cost: 59.93
Connected: 157648 → 1 | Length: 5411.2 m | Cost: 143.23
Connected: 157648 → 2 | Length: 7091.7 m | Cost: 189.30
Connected: 157648 → 3 | Length: 4856.1 m | Cost: 129.35
Connected: 157648 → 4 | Length: 3481.7 m | Cost: 94.99
Connected: 157648 → 5 | Length: 3151.6 m | Cost: 90.80
Connected: 157648 → 6 | Length: 4250.9 m | Cost: 118.28
Connected: 157648 → 7 | Length: 2861.1 m | Cost: 79.48
Connected: 157648 → 8 | Length: 2643.8 m | Cost: 76.08
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m | Cost: 86.26
Connected: 157648 → 11 | Length: 5987.9 m | Cost: 161.71
Connected: 157648 → 12 | Length: 3133.2 m | Cost: 88.31
Connected: 157648 → 13 | Length: 5332.2 m | Cost: 145.31
Connected: 157648 → 14 | Length: 6446.6 m | Cost: 169.11
Connected: 157648 → 15 | Length: 6200.1 m | Cost: 162.95
Connected: 157648 → 16 | Length: 4979.6 m | Cost: 134.47
Connected: 157648 → 17 

In [10]:
for area in target_area_types:
    for factor in likelihood_cols:
        for delta in [-1, +1]:
            scenario_name = f"{area} - {factor} {'+' if delta > 0 else ''}{delta}"

            # Deepcopy van df om origineel te behouden
            df_mod = df.copy()

            # Pas de likelihoodwaarde alleen aan voor het geselecteerde gebiedstype
            affected_rows = df_mod["area_type"] == area
            df_mod.loc[affected_rows, factor] = (
                df_mod.loc[affected_rows, factor] + delta
            ).clip(lower=1, upper=5)  # Ordinale grenzen bewaken

            # Herbereken risico’s
            R_f_total, R_p_total = 0, 0
            for i in range(1, 6):
                df_mod[f"R_if_{i}"] = df_mod["Sf"] * df_mod[f"L{i}"]
                df_mod[f"R_ip_{i}"] = df_mod["Sp"] * df_mod[f"L{i}"]
                R_f_total += df_mod[f"R_if_{i}"]
                R_p_total += df_mod[f"R_ip_{i}"]

            df_mod["R_f"] = R_f_total
            df_mod["R_p"] = R_p_total
            df_mod["R_s"] = df_mod["Ss"]

            # Normalisatie
            df_mod["R_f_norm"] = (df_mod["R_f"] - df_mod["R_f"].min()) / (df_mod["R_f"].max() - df_mod["R_f"].min())
            df_mod["R_p_norm"] = (df_mod["R_p"] - df_mod["R_p"].min()) / (df_mod["R_p"].max() - df_mod["R_p"].min())
            df_mod["R_s_norm"] = (df_mod["R_s"] - df_mod["R_s"].min()) / (df_mod["R_s"].max() - df_mod["R_s"].min())

            # Loop over alpha’s
            for alpha in alpha_values:
                # Gewogen risicoscore toepassen
                df_mod["risk"] = (
                    0.4 * df_mod["R_f_norm"] + 
                    0.3 * df_mod["R_p_norm"] + 
                    0.3 * df_mod["R_s_norm"]
                )

                # Mapping area_type -> risk voor edges
                etype_to_risk = dict(zip(df_mod["area_type"], df_mod["risk"]))
                for u, v, data in G.edges(data=True):
                    etype = data.get("etype")
                    if etype in etype_to_risk:
                        data["risk"] = etype_to_risk[etype]
                    if etype == "postnl_connector":
                        data["risk"] = 0.0

                # Run planner
                connected, _ = connect_distribution_to_postnl(G, alpha=alpha, method="dijkstra")

                # Resultaatanalyse
                total_risk = 0
                total_energy = 0
                etype_usage = []
                total_length = 0
                for conn in connected:
                    _, _, _, _, path_edges, etypes = conn
                    etype_usage.extend(etypes)
                    for geom, u, v in path_edges:
                        edge = G.get_edge_data(u, v)
                        total_risk += edge.get("risk", 0)
                        total_energy += edge.get("energy", 0)
                        total_length += geom.length

                sa_results.append({
                    "scenario": scenario_name,
                    "alpha": alpha,
                    "n_connected": len(connected),
                    "total_risk": total_risk,
                    "total_energy": total_energy,
                    "total_length": total_length,
                    "etype_usage": etype_usage,
                })


Distribution points: 1
PostNL points: 64
Alpha: 0.0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m | Cost: 59.93
Connected: 157648 → 1 | Length: 5411.2 m | Cost: 143.23
Connected: 157648 → 2 | Length: 7091.7 m | Cost: 189.30
Connected: 157648 → 3 | Length: 4856.1 m | Cost: 129.35
Connected: 157648 → 4 | Length: 3481.7 m | Cost: 94.99
Connected: 157648 → 5 | Length: 3151.6 m | Cost: 90.80
Connected: 157648 → 6 | Length: 4250.9 m | Cost: 118.28
Connected: 157648 → 7 | Length: 2861.1 m | Cost: 79.48
Connected: 157648 → 8 | Length: 2643.8 m | Cost: 76.08
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m | Cost: 86.26
Connected: 157648 → 11 | Length: 5987.9 m | Cost: 161.71
Connected: 157648 → 12 | Length: 3133.2 m | Cost: 88.31
Connected: 157648 → 13 | Length: 5332.2 m | Cost: 145.31
Connected: 157648 → 14 | Length: 6446.6 m | Cost: 169.11
Connected: 157648 → 15 | Length: 6200.1 m | Cost: 162.95
Connected: 157648 → 16 | Length: 4979.6 m | Cost: 134.47
Connected: 157648 → 17 

In [11]:
df_sa_likelihood = pd.DataFrame(sa_results)

# Bereken verdeling etypes (optioneel)
from collections import Counter
df_sa_likelihood["etype_counts"] = df_sa_likelihood["etype_usage"].apply(lambda x: dict(Counter(x)))
df_sa_likelihood.drop(columns=["etype_usage"], inplace=True)

df_sa_likelihood

,scenario,alpha,n_connected,total_risk,total_energy,total_length,etype_counts
0,Base,0.0,63,86473.610645,7158.149730,262881.589216,"{'postnl_connector': 126, 'Industrial zones': ..."
1,Base,0.5,63,20727.883605,9459.284719,353384.188777,"{'postnl_connector': 126, 'Industrial zones': ..."
2,Base,1.0,63,20288.837160,10691.464475,402265.378998,"{'postnl_connector': 126, 'Industrial zones': ..."
3,Meadows and open grass - L1 -1,0.0,63,4468.202109,7158.149730,262881.589216,"{'postnl_connector': 126, 'Industrial zones': ..."
4,Meadows and open grass - L1 -1,0.5,63,1038.829156,9621.377418,359786.696729,"{'postnl_connector': 126, 'Industrial zones': ..."
...,...,...,...,...,...,...,...
148,Tracks and rural access roads - L5 -1,0.5,63,1053.485856,9464.179855,354067.194208,"{'postnl_connector': 126, 'Industrial zones': ..."
149,Tracks and rural access roads - L5 -1,1.0,63,1049.756576,10694.881556,402402.062245,"{'postnl_connector': 126, 'Industrial zones': ..."
150,Tracks and rural access roads - L5 +1,0.0,63,4522.275682,7158.149730,262881.589216,"{'postnl_connector': 126, 'Industrial zones': ..."
151,Tracks and rural access roads - L5 +1,0.5,63,1115.150868,9444.021576,352773.663021,"{'postnl_connector': 126, 'Industrial zones': ..."


In [12]:
df_sa_likelihood['avg_length_m'] = df_sa_likelihood['total_length'] / df_sa_likelihood['n_connected']

In [ ]:
df_sa_likelihood.to_csv("output/sa2_likelihood_results.csv", index=False)

In [ ]:
# Selecteer base case
df_base = df_sa_likelihood[df_sa_likelihood["scenario"] == "Base"].copy()

# Maak een dict zodat we makkelijk per alpha kunnen refereren
base_values = df_base.set_index("alpha")[["total_risk", "total_length"]].to_dict(orient="index")


In [ ]:
def compute_deviation(row):
    base = base_values[row['alpha']]
    risk_dev = (row['total_risk'] - base['total_risk']) / base['total_risk'] * 100
    length_dev = (row['total_length'] - base['total_length']) / base['total_length'] * 100
    return pd.Series({"risk_dev": risk_dev, "length_dev": length_dev})

# Alleen voor niet-base scenario's
df_sensitivity = df_sa_likelihood[df_sa_likelihood["scenario"] != "Base"].copy()

df_sensitivity[["risk_dev", "length_dev"]] = df_sensitivity.apply(compute_deviation, axis=1)


In [ ]:
sa2 = pd.read_csv("/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/sa2_likelihood_results.csv")

In [ ]:
sa2.drop(columns=["etype_counts", "n_connected"], inplace=True)

In [ ]:
df = sa2

In [ ]:
sa2

In [ ]:
# Eerst baseline extracten voor vergelijking
baseline = df[df['scenario'] == 'Base'].set_index('alpha')[['total_risk', 'total_energy', 'total_length']]

# Functie om scenario-omschrijving netjes te splitsen
def parse_scenario(s):
    if s == 'Base':
        return 'Base', '', ''
    else:
        parts = s.split(' - ')
        area_type = parts[0]
        factor = parts[1] if len(parts) > 1 else ''
        shift = parts[2] if len(parts) > 2 else ''
        return area_type, factor, shift

# Nieuwe kolommen aanmaken
df[['Area Type', 'Perturbed Factor', 'Shift']] = df['scenario'].apply(lambda s: pd.Series(parse_scenario(s)))

# Alleen de perturbation rows
df_perturb = df[df['scenario'] != 'Base']

# Lege lijst voor resultaten
results = []

# Over alle perturbation scenario's loopen
for idx, row in df_perturb.iterrows():
    alpha = row['alpha']
    base_row = baseline.loc[alpha]
    
    delta_risk = (row['total_risk'] - base_row['total_risk']) / base_row['total_risk'] * 100
    delta_energy = (row['total_energy'] - base_row['total_energy']) / base_row['total_energy'] * 100
    delta_length = (row['total_length'] - base_row['total_length']) / base_row['total_length'] * 100
    
    results.append({
        'Area Type': row['Area Type'],
        'Perturbed Factor': row['Perturbed Factor'],
        'Shift': row['Shift'],
        'Alpha': alpha,
        'Δ Total Risk (%)': round(delta_risk, 2),
        'Δ Total Energy (%)': round(delta_energy, 2),
        'Δ Total Length (%)': round(delta_length, 2),
    })

# DataFrame bouwen
df_summary = pd.DataFrame(results)

# Even sorteren voor netheid
df_summary = df_summary.sort_values(by=['Area Type', 'Perturbed Factor', 'Shift', 'Alpha'])

# Output
print(df_summary.head(10))

# Eventueel opslaan als CSV om makkelijk in thesis tabel te plakken:
df_summary.to_csv('output/sa2.csv', index=False)

In [ ]:
# Exporteren naar LaTeX
with open('output/sa2_likelihood.tex', 'w') as f:
    f.write(df_summary.to_latex(index=False, escape=False))

In [ ]:
# Groeperen per Area Type en Alpha
summary = df_summary.groupby(['Area Type', 'Alpha']).agg({
    'Δ Total Risk (%)': ['mean', 'max'],
    'Δ Total Energy (%)': 'mean',
    'Δ Total Length (%)': 'mean',
}).reset_index()

# Kolomnamen netter maken
summary.columns = ['Area Type', 'Alpha',
                   'Mean Δ Risk (%)', 'Max Δ Risk (%)',
                   'Mean Δ Energy (%)', 'Mean Δ Length (%)']

# Eventueel afronden
summary = summary.round(2)

# Sorteren voor leesbaarheid
summary = summary.sort_values(by=['Area Type', 'Alpha'])

# Exporteren voor thesis tabel
summary.to_csv('output/sa2_summary.csv', index=False)


In [ ]:
summary

In [ ]:
# Exporteren naar LaTeX
with open('output/sa2_likelihood_summary.tex', 'w') as f:
    f.write(summary.to_latex(index=False, column_format="llrrrr", escape=False))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_sensitivity, x="alpha", y="length_dev")
plt.title("Route length deviation (%) per alpha")
plt.ylabel("Length deviation (%)")
plt.show()

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_sensitivity, x="alpha", y="risk_dev")
plt.title("Total risk deviation (%) per alpha")
plt.ylabel("Risk deviation (%)")
plt.show()
